## Project Requirements:
Scrape profiles and get:
1. the number of posts
2. the follower count
3. number of likes on most liked post
4. numbner of likes

## Follow Up
Save each profile in its own data frame with each row corresponding to a post <br>
export as a excel file with each profile as a worksheet <br>
find a way to edit so that the sheet includes some global account variables (ex. number of followers) outside the dataframe

In [1]:
from instascrape import Profile, scrape_posts, Post
from selenium.webdriver import Chrome
from datetime import datetime
import pandas as pd
import numpy as np
import pickle
from openpyxl.workbook.child import INVALID_TITLE_REGEX
from openpyxl import workbook
import re
from utils import check_valid_username, write_to_excel, get_top_post, create_post_df
from pathlib import Path
from tqdm import tqdm

In [2]:
cwd = Path.cwd()

#mac
#chrome_driver_path = cwd / 'chromedriver'

#windows
chrome_driver_path = cwd / 'chromedriver.exe'
driver = Chrome(chrome_driver_path)


In [3]:
headers = {
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36 Edg/87.0.664.57",
    "cookie": "sessionid=7320119797%3AiFhQYSC7AxwAcY%3A17"
}

In [4]:
users = pd.read_csv("Social_Media_Study_Handles.csv")
users = users.reindex(columns=[*users.columns, 'follower_count', 'number_of_posts'])

#Specifying datatype for each columns
users['School'] = users['School'].astype(object)
users['IG Username'] = users['IG Username'].astype(object)
users['follower_count'] = users['follower_count'].astype('Int64')
users['number_of_posts'] = users['number_of_posts'].astype('Int64')

# users = users.reindex(columns=[*users.columns, 'follower_count', 'number_of_posts', 'top_post_likes', 
# 'top_post_date_posted', 'top_post_url', 'first_post_likes', 'first_post_date_posted', 'first_post_url'])
# This code is no longer userful since we are storing each account's posts in its own dataframe
# users['top_post_likes'] = users['top_post_likes'].astype('Int64')
# users['top_post_date_posted'] = users['top_post_date_posted'].astype('datetime64[ns]')
# users['top_post_url'] = users['top_post_url'].astype(object)
# users['first_post_likes'] = users['first_post_likes'].astype('Int64')
# users['first_post_date_posted'] = users['first_post_date_posted'].astype('datetime64[ns]')
# users['first_post_url'] = users['first_post_url'].astype(object)

In [5]:
#test purposes
# users = users.head(5).append(users.iloc[89])

#List of dataframes representing the posts of each user
users_df = []

#indicies of all users with more than 200 posts
index_greater_200 = []

#indicies of all users with less than 200 posts
index_200_or_less = []

#Index of null
index_null = []

for i in tqdm(range(0, len(users))):
    #Putting a try catch statement here because there seems to be some errors that could be fixed with an extra try
    attmepts_allowed = 3
    for attmept in range(attmepts_allowed):
        try:
            if (not pd.isnull(users.iloc[i]['IG Username'])):
                #Scrape the profile
                print(users.iloc[i]['IG Username'])
                profile = Profile(users.iloc[i]['IG Username'])
                profile.scrape(headers=headers)

                #Adding profile datapoints to dataframe
                users.at[i, 'follower_count'] = profile.followers
                users.at[i, 'number_of_posts'] = profile.posts

                #Scraping Posts

                posts = profile.get_posts(webdriver=driver)
                print(len(posts))

                #account has less than 200 posts
                if (len(posts) <= 200):
                    index_200_or_less.append(i)
                    scraped_posts, unscraped = scrape_posts(posts, webdriver=driver, silent=False, headers=headers, pause=7)
                    
                    #Appending dataframe from account to each 
                    users_df.append(create_post_df(scraped_posts))
                
                else:
                    print("{} with username {} has {} posts, more than 200".format(users.iloc[i]['School'], users.iloc[i]['IG Username'], len(posts)))
                    
                    #no matter what, we want to keep the dimensions of user_df the same as users
                    users_df.append(create_post_df([]))
                    index_greater_200.append(i)
                        
            # if no username, we're just going to add an empty dataframe
            else:
                users_df.append(create_post_df([]))
                index_null.append(i)

        #Error, let's try again
        except Exception as e:
            print("an error has occured on {}".format(users.iloc[i]['IG Username']))
            print(e)
            #Adding empty dataframe to list anyways just to keep the indicies matching

        #No errors, let's exit the loop
        else:
            break

    #Did we finish the for loop, that means all attempts failed
    else:
        #Adding empty dataframe to list anyways just to keep the indicies matching
        users_df.append(create_post_df([]))
        print("all attempts failed")
    
    #Writing test dataframes to pickle object for easier debugging in the future
    if (i % 10 == 0 and i != 0):
        with open('test_dataframes_{}-{}.obj'.format(i-10, i), 'wb') as f:
            pickle.dump((users_df, users), f)
        
        
        
        #old code from just getting top and first posts
        # top_post = get_top_post(scraped_posts)

        # #Post are scraped from most recent to oldest, therefore, the earliest post will be the last one scraped in theory
        # first_post = scraped_posts[-1]

        # users.at[i, 'top_post_likes'] = top_post.likes
        # users.at[i, 'top_post_date_posted'] = datetime.fromtimestamp(top_post.timestamp)
        # users.at[i, 'top_post_url'] = "instagram.com/p/{}".format(top_post.shortcode)
        # users.at[i, 'first_post_likes'] = first_post.likes
        # users.at[i, 'first_post_date_posted'] = datetime.fromtimestamp(first_post.timestamp)
        # users.at[i, 'first_post_url'] = "instagram.com/p/{}".format(first_post.shortcode)


  0%|          | 0/121 [00:00<?, ?it/s]

willseyeresidents
33
0: CDABF2hhEhP - 2020-07-23 17:49:11
1: CC6_LwDhnTu - 2020-07-21 18:56:19
2: CC4Kk8ZBGIc - 2020-07-20 16:38:09
3: CCtc94uh_SW - 2020-07-16 12:47:12
4: CCqrp49B5j1 - 2020-07-15 10:57:49
5: CCjOYsGBwWO - 2020-07-12 13:27:23
6: CCecas4hdY- - 2020-07-10 16:53:47
7: CCHSdAehmf7 - 2020-07-01 17:04:11
8: CB3xPaehw43 - 2020-06-25 16:25:22
9: CBy5lzbBlq8 - 2020-06-23 19:02:08
10: CBwQp8qhq3y - 2020-06-22 18:25:57
11: CBtTXidBM4k - 2020-06-21 14:51:56
12: CBqW91ohOfD - 2020-06-20 11:25:39
13: CBoCRLXhQex - 2020-06-19 13:46:18
14: CBlvolqAn4t - 2020-06-18 16:25:00
15: CBjLzmHBJxE - 2020-06-17 16:33:27
16: CANgGIPAQc0 - 2020-05-15 09:56:03
17: CALFcpLAZq1 - 2020-05-14 11:24:43
18: B_lFRQnBz3j - 2020-04-29 17:12:01
19: B-sAg1yhXLQ - 2020-04-07 13:13:45
20: B9Z5wFmBtvD - 2020-03-06 14:56:56
21: B9HGomcBRkn - 2020-02-28 07:43:57
22: B8nbgg2hDsc - 2020-02-16 00:30:39
23: B7yyPaVB9wF - 2020-01-26 13:49:32
24: B6UcX8CB3J_ - 2019-12-20 22:29:51
25: B6TRZBrBHoW - 2019-12-20 11:34:38
2

  1%|          | 1/121 [04:32<9:04:58, 272.49s/it]

32: B4BZYIpAuYT - 2019-10-24 20:55:18
bascompalmereye


  2%|▏         | 2/121 [05:15<4:32:11, 137.24s/it]

...

10: CG2_PENn10B - 2020-10-27 15:45:23
cwru_med
12
0: CVdckQcLEQC - 2021-10-25 13:32:20
1: CVQyEiflmoI - 2021-10-20 15:30:07
2: CVOJSE1Fvaw - 2021-10-19 14:55:13
3: CVB9COAIWpz - 2021-10-14 21:17:19
4: CU-Mf7nrnt2 - 2021-10-13 10:15:29
5: CUpgL3NLTsd - 2021-10-05 09:23:27
6: CUIDHC5Lp9X - 2021-09-22 09:34:27
7: CT7TpgSr-3x - 2021-09-17 10:48:50
8: CTPogA7HLxo - 2021-08-31 11:44:28
9: CTALDPnnXu_ - 2021-08-25 11:38:31
10: CSziwETHTyY - 2021-08-20 13:55:29


 50%|█████     | 61/121 [2:22:27<1:23:56, 83.94s/it]

11: CShAUFmHnkr - 2021-08-13 09:08:15
rwjms


 50%|█████     | 61/121 [2:22:36<2:20:16, 140.27s/it]


IndexError: list index out of range

In [6]:
profile = Profile("rwjms")
profile.scrape(headers=headers)

In [ ]:
users_df_np = np.array(users_df, dtype=object)
write_to_excel(users_df[index_200_or_less], users.iloc[index_200_or_less])

TypeError: list indices must be integers or slices, not list

In [66]:
len(users_df)

80

In [62]:
index_200_less_and_null = index_200_or_less.copy()
index_200_less_and_null.extend(users.loc[pd.isna(users["IG Username"])].index.tolist())
index_200_less_and_null

[0,
 2,
 3,
 4,
 5,
 6,
 8,
 9,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 23,
 24,
 27,
 29,
 30,
 31,
 32,
 33,
 37,
 39,
 40,
 41,
 42,
 43,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 58,
 59,
 62,
 63,
 65,
 66,
 67,
 69,
 70,
 74,
 75,
 76,
 77,
 78,
 79,
 81,
 85,
 86,
 87,
 90,
 92,
 94,
 99,
 102,
 106,
 107,
 111,
 115,
 117,
 120,
 83,
 89,
 95,
 100,
 104,
 105,
 108,
 110,
 112,
 83,
 89,
 95,
 100,
 104,
 105,
 108,
 110,
 112,
 83,
 89,
 95,
 100,
 104,
 105,
 108,
 110,
 112]

In [54]:
index_200_or_less.copy()

[0,
 2,
 3,
 4,
 5,
 6,
 8,
 9,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 23,
 24,
 27,
 29,
 30,
 31,
 32,
 33,
 37,
 39,
 40,
 41,
 42,
 43,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 58,
 59,
 62,
 63,
 65,
 66,
 67,
 69,
 70,
 74,
 75,
 76,
 77,
 78,
 79,
 81,
 85,
 86,
 87,
 90,
 92,
 94,
 99,
 102,
 106,
 107,
 111,
 115,
 117,
 120,
 83,
 89,
 95,
 100,
 104,
 105,
 108,
 110,
 112]

In [59]:
users.loc[pd.isna(users["IG Username"])].index.tolist()

[83, 89, 95, 100, 104, 105, 108, 110, 112]

In [61]:
test_list.extend(users.loc[pd.isna(users["IG Username"])].index.tolist())
test_list

[0,
 2,
 3,
 4,
 5,
 6,
 8,
 9,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 23,
 24,
 27,
 29,
 30,
 31,
 32,
 33,
 37,
 39,
 40,
 41,
 42,
 43,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 58,
 59,
 62,
 63,
 65,
 66,
 67,
 69,
 70,
 74,
 75,
 76,
 77,
 78,
 79,
 81,
 85,
 86,
 87,
 90,
 92,
 94,
 99,
 102,
 106,
 107,
 111,
 115,
 117,
 120,
 83,
 89,
 95,
 100,
 104,
 105,
 108,
 110,
 112,
 83,
 89,
 95,
 100,
 104,
 105,
 108,
 110,
 112]

In [42]:
test_list = index_200_or_less

In [44]:
len(test_list)

80

In [15]:
len(users_df)

80

In [16]:
users.iloc[index_200_or_less].shape

(71, 4)

In [11]:
def write_to_excel(user_post_dfs, users_df):
    if (len(user_post_dfs) != users_df.shape[0]):
        raise ValueError("length of the posts dataframes and user dataframes are mismatched, there are {} posts dataframes and {} rows in the user dataframe".format(
            len(user_post_dfs), users_df.shape[0]))

    with pd.ExcelWriter('User Data.xlsx', engine='openpyxl') as writer:
        for i in range(0, len(user_post_dfs)):
            # this regex statement replaces excel's invalid characters with and underscore
            title = re.sub(INVALID_TITLE_REGEX, '_', users_df.at[i, 'School'])
            users_df.iloc[[i]].to_excel(writer, sheet_name=title, index=False)
            user_post_dfs[i].to_excel(
                writer, startrow=3, sheet_name=title, index=False)

In [13]:
#Writing test dataframes to pickle object for easier debugging in the future
with open('test_dataframes.obj', 'wb') as f:
    pickle.dump((users_df, users), f)

In [3]:
with open('test_dataframes.obj', 'rb') as f:
    loaded = pickle.load(f)

users_df = loaded[0]
users = loaded[1]
users = users.reset_index()